<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD; 
    color: black;"> <img style="float: left; padding-right: 10px;" src="https://storage.googleapis.com/dlops-content/public/univailogo.jpeg" height="50px"> <a href='https://welcome.univ.ai/AcceleratedMasters/ai5-c1/' target='_blank'><strong><font color="#324bd9">AI-5: Productionizing AI (MLOps)</font></strong></a></h1>

# **<font color="#324bd9">Exercise 4 - Language Models</font>**

**Univ.AI**<br/>
**August 2021**<br/>
**Instructors:**<br/>
Pavlos Protopapas, Shivas Jayaram

<hr style="height:2pt">

## **<font color="#324bd9">Setup Notebook</font>**

**Copy & setup Colab**

1) Select "File" menu and pick "Save a copy in Drive"

**Installs**

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 30.4 MB/s 
     |████████████████████████████████| 50 kB 4.8 MB/s 
     |████████████████████████████████| 636 kB 31.8 MB/s 
     |████████████████████████████████| 3.3 MB 33.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


**Imports**

In [ ]:
import os
import requests
import zipfile
import tarfile
import shutil
import math
import json
import time
import sys
import string
import re
import numpy as np
import pandas as pd
from glob import glob
import collections
import unicodedata
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.utils.layer_utils import count_params

# sklearn
from sklearn.model_selection import train_test_split

# NLTK
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

# Transformers
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, GPT2Config

In [ ]:
# download nltk's punkt sentence tokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**Verify Setup**

In [ ]:
# Enable/Disable Eager Execution
# Reference: https://www.tensorflow.org/guide/eager
# TensorFlow's eager execution is an imperative programming environment that evaluates operations immediately, 
# without building graphs

#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()

print("tensorflow version", tf.__version__)
print("keras version", tf.keras.__version__)
print("Eager Execution Enabled:", tf.executing_eagerly())

# Get the number of replicas 
strategy = tf.distribute.MirroredStrategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

devices = tf.config.experimental.get_visible_devices()
print("Devices:", devices)
print(tf.config.experimental.list_logical_devices('GPU'))

print("GPU Available: ", tf.config.list_physical_devices('GPU'))
print("All Physical Devices", tf.config.list_physical_devices())

# Better performance with the tf.data API
# Reference: https://www.tensorflow.org/guide/data_performance
AUTOTUNE = tf.data.experimental.AUTOTUNE

tensorflow version 2.6.0
keras version 2.6.0
Eager Execution Enabled: True
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of replicas: 1
Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]
GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
All Physical Devices [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


**Utils**

In [ ]:
def download_file(packet_url, base_path="", extract=False, headers=None):
  if base_path != "":
    if not os.path.exists(base_path):
      os.mkdir(base_path)
  packet_file = os.path.basename(packet_url)
  with requests.get(packet_url, stream=True, headers=headers) as r:
      r.raise_for_status()
      with open(os.path.join(base_path,packet_file), 'wb') as f:
          for chunk in r.iter_content(chunk_size=8192):
              f.write(chunk)
  
  if extract:
    if packet_file.endswith(".zip"):
      with zipfile.ZipFile(os.path.join(base_path,packet_file)) as zfile:
        zfile.extractall(base_path)
    else:
      packet_name = packet_file.split('.')[0]
      with tarfile.open(os.path.join(base_path,packet_file)) as tfile:
        tfile.extractall(base_path)

## **Mini BERT**
---

In this exercise, you are going to be working with some text related to machine & deep learning. The data is a very small set of defintions pulled from wikipedia. The primary goal will be to understand BERT(Bidirectional Encoder Representations from Transformers) architecture.

**The Task:** Build a masked language model using a simplified version of the BERT. 


**Language Model**: 

A model that understands language and how words appear in context to one another. The model is trained using unsupervised approaches such as next word prediction in a sentence or next sentence prediction.


**Review BERT**:

* Masked Language Model
* Made up of only the Encoder with stacked transformer blocks
* Bidirectional language
* Good for fill in the blanks

### **Dataset**

#### **Download Dataset**

Download the datasets to colab.

In [ ]:
start_time = time.time()
download_file("https://github.com/dlops-io/datasets/releases/download/v1.0/deep_learning_terms.txt", base_path="datasets", extract=False)
execution_time = (time.time() - start_time)/60.0
print("Download execution time (mins)",execution_time)

Download execution time (mins) 0.007071165243784586


#### **Load Data**

* Read-in data as lists.

In [ ]:
data_text_file = os.path.join("datasets","deep_learning_terms.txt")
data_text = []
with open(data_text_file) as file:
  for line in file:
    data_text.append(line)

print("data_text len:", len(data_text))

# Get sentences
sentences = []
for text in data_text:
  sentences.extend(sent_tokenize(text))

# Add a few DEMO sentence
sentences.append("pavlos taught positional encoding to shivas")
print("sentences len:", len(sentences))

data_text len: 20
sentences len: 183


#### **View Text**

Let's take a look at the data.

In [ ]:
display(sentences[:10])

['Deep learning also known as deep structured learning is part of a broader family of machine learning methods based on artificial neural networks with representation learning.',
 'Learning can be supervised, semi-supervised or unsupervised.',
 'Deep-learning architectures such as deep neural networks, deep belief networks, recurrent neural networks and convolutional neural networks have been applied to fields including computer vision, machine vision, speech recognition, natural language processing, audio recognition, social network filtering, machine translation, bioinformatics, drug design, medical image analysis, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.',
 'Artificial neural networks (ANNs) were inspired by information processing and distributed communication nodes in biological systems.',
 'ANNs have various differences from biological brains.',
 'Specifically, neural networks

### **Build Data Pipelines**

#### **Text Vectorization**

In [ ]:
MASK_TOKEN = "[mask]"
max_len = max([len(x.split()) for x in sentences])
print("max_len",max_len)

def standardize_text(input_text):
  # Convert to lowercase
  output_text = tf.strings.lower(input_text) 
  return tf.strings.regex_replace(
        output_text, "[%s]" % re.escape("!#$%&'()*+,-./:;<=>?@\^_`{|}~"), ""
    )

text_data = tf.data.Dataset.from_tensor_slices(sentences)

# Initialize Text Vectorizer
text_vectorizer = TextVectorization(
    standardize=standardize_text,
    output_mode="int",
    output_sequence_length=max_len
)

# Generate Text Vector
text_vectorizer.adapt(sentences)

# Get Vocabulary
vocabulary = text_vectorizer.get_vocabulary()
vocabulary_size = len(vocabulary)
# Add the mask token
vocabulary = vocabulary[2 : vocabulary_size - 1] + [MASK_TOKEN]
text_vectorizer.set_vocabulary(vocabulary)
vocabulary = text_vectorizer.get_vocabulary()
vocabulary_size = len(vocabulary)
print("Vocabulary Size:",vocabulary_size)
print(vocabulary[-1])

# Generate word index
word_index = dict(zip(vocabulary, range(vocabulary_size)))
index_word = dict(zip(range(vocabulary_size), vocabulary))

print("vocabulary:",len(vocabulary),vocabulary)
print("word_index:",word_index)

# Get mask token id for masked language model
mask_token_id = word_index[MASK_TOKEN]
print("Mask token Id:", mask_token_id)

max_len 66
Vocabulary Size: 1130
[mask]
vocabulary: 1130 ['', '[UNK]', 'the', 'of', 'a', 'to', 'and', 'is', 'in', 'learning', 'that', 'as', 'or', 'data', 'can', 'with', 'neural', 'networks', 'be', 'language', 'such', 'an', 'are', 'machine', 'image', 'from', 'for', 'network', 'this', 'model', 'on', 'layers', 'it', 'artificial', 'training', 'deep', 'by', 'also', 'recognition', 'layer', 'computer', 'used', 'natural', 'input', 'have', 'which', 'translation', 'not', 'neurons', 'where', 'process', 'human', 'between', 'words', 'vision', 'supervised', 'speech', 'processing', 'field', 'feature', 'systems', 'recurrent', 'output', 'nlg', 'more', 'its', 'intelligence', 'each', 'understanding', 'tasks', 'signal', 'representation', 'other', 'one', 'labeled', 'known', 'into', 'images', 'different', 'biological', 'algorithms', 'when', 'visual', 'unsupervised', 'they', 'than', 'segmentation', 'produce', 'models', 'lstm', 'information', 'but', 'use', 'through', 'system', 'set', 'sequences', 'semisupervi

In [ ]:
# Convert input text to tokens
all_data_tokens = text_vectorizer(sentences)
print("Input text:\n")
print(sentences[0])
print(sentences[1])
print(sentences[3])
print("all_data_tokens shape:",all_data_tokens.shape)
print("Tokenized text:", all_data_tokens[:5,:20])

Input text:

Deep learning also known as deep structured learning is part of a broader family of machine learning methods based on artificial neural networks with representation learning.
Learning can be supervised, semi-supervised or unsupervised.
Artificial neural networks (ANNs) were inspired by information processing and distributed communication nodes in biological systems.
all_data_tokens shape: (183, 66)
Tokenized text: tf.Tensor(
[[  35    9   37   75   11   35  192    9    7  150    3    4 1048  879
     3   23    9  101  109   30]
 [   9   14   18   55   97   12   83    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [ 961  428   20   11   35   16   17   35  420   17   61   16   17    6
   244   16   17   44  254  170]
 [  33   16   17  256  178  220   36   90   57    6  934 1013  206    8
    79   60    0    0    0    0]
 [ 256   44  268  394   25   79  251    0    0    0    0    0    0    0
     0    0    0    0    0    0]], shape=(5, 20), dtype=int64)


#### **Generate Training Data**

For the training we need inputs and lables but we only have input texts. In the lecture we learnt that language models are trained in a semi supervised way where we generate inputs and labels from the input text. 

<br>

To generate inputs and lables for training we will mask random words from the input text. Then our labels will be the masked text. The model will be trained to predict the masked word

Steps for generating data:
* 15% random tokens are take
* 90% of these are masked
* the other 10% is set to random tokens

In [ ]:
# Since we have a very small dataset, we will just replicate the data
encoded_texts = all_data_tokens.numpy()
print(encoded_texts.shape)
encoded_texts = np.vstack([encoded_texts]*50)
print(encoded_texts.shape)

(183, 66)
(9150, 66)


In [ ]:
# 15% BERT masking
inp_mask = np.random.rand(*encoded_texts.shape) < 0.15
print("inp_mask:", inp_mask.shape)
#print("inp_mask:",inp_mask[0,:25])

# Exclude masking special tokens
inp_mask[encoded_texts <= 2] = False

# Set targets to -1 by default, it means ignore
labels = -1 * np.ones(encoded_texts.shape, dtype=int)

# Set labels for masked tokens
labels[inp_mask] = encoded_texts[inp_mask]

# Prepare input
encoded_texts_masked = np.copy(encoded_texts)

# Set input to [MASK] which is the last token for the 90% of tokens
# This means leaving 10% unchanged
inp_mask_2mask = inp_mask & (np.random.rand(*encoded_texts.shape) < 0.90)
encoded_texts_masked[
    inp_mask_2mask
] = mask_token_id  # mask token is the last in the dict

# Set 10% to a random token
inp_mask_2random = inp_mask_2mask & (np.random.rand(*encoded_texts.shape) < 1 / 9)
encoded_texts_masked[inp_mask_2random] = np.random.randint(
    3, mask_token_id, inp_mask_2random.sum()
)

# Prepare sample_weights to pass to .fit() method
sample_weights = np.ones(labels.shape)
sample_weights[labels == -1] = 0

# y_labels would be same as encoded_texts i.e input tokens
y_labels = np.copy(encoded_texts)

print("encoded_texts_masked:",encoded_texts_masked.shape)
print(encoded_texts_masked[:5,:10])
print("y_labels:",y_labels.shape)
print(y_labels[:5,:10])
print("sample_weights:",sample_weights.shape)
print(sample_weights[:5,:10])

inp_mask: (9150, 66)
encoded_texts_masked: (9150, 66)
[[  35    9 1129   75   11   35  192    9    7  150]
 [   9   14   18   55   97   12   83    0    0    0]
 [ 961  428   20   11 1129   16   17 1129  420   17]
 [  33   16 1129 1129 1129  220 1129   90   57    6]
 [ 256   44  268  394   25   79  251    0    0    0]]
y_labels: (9150, 66)
[[ 35   9  37  75  11  35 192   9   7 150]
 [  9  14  18  55  97  12  83   0   0   0]
 [961 428  20  11  35  16  17  35 420  17]
 [ 33  16  17 256 178 220  36  90  57   6]
 [256  44 268 394  25  79 251   0   0   0]]
sample_weights: (9150, 66)
[[0. 0. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 0. 1. 1. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


#### **Create TF Datasets**

In [ ]:
BATCH_SIZE = 64
TRAIN_SHUFFLE_BUFFER_SIZE = encoded_texts.shape[0]

# Create TF Dataset
train_data = tf.data.Dataset.from_tensor_slices((encoded_texts_masked, y_labels, sample_weights))

#############
# Train data
#############
train_data = train_data.shuffle(buffer_size=TRAIN_SHUFFLE_BUFFER_SIZE)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

print("train_data",train_data)

train_data <PrefetchDataset shapes: ((None, 66), (None, 66), (None, 66)), types: (tf.int64, tf.int64, tf.float64)>


### **Build Mini BERT**

#### **Positional Encoding**

In [ ]:
def generate_positional_encoding(max_length, model_size):
    pos_enc = np.array(
        [
            [pos / np.power(10000, 2 * (j // 2) / model_size) for j in range(model_size)]
            if pos != 0
            else np.zeros(model_size)
            for pos in range(max_length)
        ]
    )
    pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2])  # dim 2i
    pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2])  # dim 2i+1
    return pos_enc

#### **Transformer Encoder Block**

Here you will build out a Transfomer Encoder block from scratch. You will use this as the building block for the mini BERT model.

<img src="https://storage.googleapis.com/public_colab_images/nlp/transformer_encoder_block.svg"/>

In [ ]:
def transfomer_encoder_block(query, key, value, embedding_dim, num_heads, ff_dim):

  # 1) Add a keras.layers.MultiHeadAttention Layer
  # Set num_heads
  # Set key_dim = embedding_dim // num_heads
  # Pass in the (query, key, value) to this layer to get the attention_output
  attention_output = keras.layers.MultiHeadAttention(num_heads,key_dim=embedding_dim//num_heads)(query,key,value)

  # 2) Add Dropout(0.1) to the attention_output
  attention_output = keras.layers.Dropout(0.1)(attention_output)

  # 3) Normalization + Residual Connection
  # Add a keras.layers.LayerNormalization with epsilon=1e-6
  # Pass in the query and attention_output to the LayerNormalization
  attention_output = keras.layers.LayerNormalization(epsilon=1e-6)(query+attention_output)

  # 4) Feedforward Layer
  # Add a Dense layer with size ff_dim and activation="relu"
  # Pass in attention_output to get the ffn_output
  # Add another Dense layer with size of embedding_dim and pass in the ffn_output
  ffn_output = keras.layers.Dense(units=ff_dim,activation='relu')(attention_output)
  ffn_output = keras.layers.Dense(units=embedding_dim,activation='relu')(ffn_output)

  # 5) Add Dropout(0.1) to the ffn_output
  ffn_output = keras.layers.Dropout(0.1)(ffn_output)

  # 6) Normalization + Residual Connection
  # Add a keras.layers.LayerNormalization with epsilon=1e-6
  # Pass in the attention_output and ffn_output to the LayerNormalization to get the final sequence_output
  sequence_output = keras.layers.LayerNormalization(epsilon=1e-6)(attention_output+ffn_output)


  return sequence_output

#### **Masked Language Model**

In [ ]:
# Loss Tracker
loss_tracker = tf.keras.metrics.Mean(name="loss")
loss_fn = keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

class MaskedLanguageModel(keras.Model):

  def train_step(self, inputs):
      if len(inputs) == 3:
          features, labels, sample_weight = inputs
      else:
          features, labels = inputs
          sample_weight = None

      with tf.GradientTape() as tape:
          predictions = self(features, training=True)
          loss = loss_fn(labels, predictions[0], sample_weight=sample_weight)

      # Compute gradients
      trainable_vars = self.trainable_variables
      gradients = tape.gradient(loss, trainable_vars)

      # Update weights
      self.optimizer.apply_gradients(zip(gradients, trainable_vars))

      # Compute our own metrics
      loss_tracker.update_state(loss, sample_weight=sample_weight)

      # Return a dict mapping metric names to current value
      return {"loss": loss_tracker.result()}

  @property
  def metrics(self):
      return [loss_tracker]

In [ ]:
def create_masked_language_bert_model(vocab_size, embedding_dim, sequence_length,num_layers, num_heads, ff_dim, enable_pe=True):
  # Model input
  model_input = keras.layers.Input(shape=(sequence_length), dtype=tf.int64)

  embeddings = keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(model_input)
  if enable_pe:
    embeddings = embeddings + generate_positional_encoding(sequence_length,embedding_dim)
  
  encoder_output = embeddings
  for i in range(num_layers):
      encoder_output = transfomer_encoder_block(encoder_output, encoder_output, encoder_output, embedding_dim,num_heads, ff_dim)

  # Output Layer
  output = keras.layers.Dense(units=vocab_size, activation="softmax")(encoder_output)

  # Create Model
  model = MaskedLanguageModel(inputs=[model_input], outputs=[output,encoder_output], name="masked_bert_model")

  return model

### **Train Mini BERT**

In [ ]:
############################
# Training Params
############################
epochs = 10
learning_rate = 0.001
embedding_dim = 256
sequence_length = max_len
num_layers = 2
num_heads = 8
ff_dim = 256

# Free up memory
K.clear_session()

# Build the model
model_w_pe = create_masked_language_bert_model(vocabulary_size,embedding_dim, sequence_length,num_layers,num_heads,ff_dim, enable_pe=True)

# Print the model architecture
print(model_w_pe.summary())

# Optimizer
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

# Compile
model_w_pe.compile(optimizer=optimizer)

# Train model
start_time = time.time()
training_results = model_w_pe.fit(
        train_data,
        epochs=epochs,
        verbose=1)
execution_time = (time.time() - start_time)/60.0
print("Training execution time (mins)",execution_time)

Model: "masked_bert_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 66)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 66, 256)      289280      input_1[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.add (TFOpLambd (None, 66, 256)      0           embedding[0][0]                  
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 66, 256)      263168      tf.__operators__.add[0][0]       
                                                                 tf.__operators__.

### **Evaluate**

Let us look at what exaclty is the language model predicting.

In [ ]:
sample_tokens = text_vectorizer(["convolutional [mask] networks have been applied to fields including computer vision"])
sample_tokens = sample_tokens.numpy()
print(sample_tokens)

[[ 244 1129   17   44  254  170    5  380  370   40   54    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0]]


In [ ]:
# Make a prediction
predictions = model_w_pe.predict(sample_tokens)

# Get the first output from the model
mask_predictions = predictions[0]
encoder_output = predictions[1]

In [ ]:
masked_index = np.where(sample_tokens == mask_token_id)
masked_index = masked_index[1]
mask_prediction = mask_predictions[0][masked_index]
print("mask_prediction:", mask_prediction)

top_indices = mask_prediction[0].argsort()[-5 :][::-1]
values = mask_prediction[0][top_indices]

def decode(tokens):
  return " ".join([index_word[t] for t in tokens if t != 0])

for i in range(len(top_indices)):
  p = top_indices[i]
  v = values[i]
  tokens = np.copy(sample_tokens[0])
  tokens[masked_index[0]] = p
  print("Prediction:",decode(tokens),"- Probability:",v, "- Mask token:",index_word[p])

mask_prediction: [[3.6319974e-09 2.8887686e-09 6.9124098e-09 ... 1.9563150e-07
  1.1716987e-08 7.8660829e-09]]
Prediction: convolutional networks networks have been applied to fields including computer vision - Probability: 0.6575382 - Mask token: networks
Prediction: convolutional neural networks have been applied to fields including computer vision - Probability: 0.112526715 - Mask token: neural
Prediction: convolutional have networks have been applied to fields including computer vision - Probability: 0.060355756 - Mask token: have
Prediction: convolutional of networks have been applied to fields including computer vision - Probability: 0.045830317 - Mask token: of
Prediction: convolutional in networks have been applied to fields including computer vision - Probability: 0.033413824 - Mask token: in


## **Generate text**
---

Generate unconditional samples using GPT2. `<|endoftext|>` is the special start token to have the model generate words on its own.

**Load Tokenizer**

In [ ]:
# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

**Load pretrained GPT2**

In [ ]:
# Model - Load pretrained GPT Language Model
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


**Generate Text**

In [ ]:
# Input text
input_text = input() # enter a prompt here

# Tokenize Input
input_ids = tokenizer.encode(input_text, return_tensors='tf')

# Generate output
outputs = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=100, 
    top_p=0.80, 
    top_k=0
)

print("Generated text:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

AI is taking
Generated text:
AI is taking in $1.2 billion in revenue this year, about 25 percent more than the $5 billion annual sales in 2011. That puts a damper on what can be considered a major free-market business, and contributes to a crisis of confidence in Microsoft as a leader in the world of software, if only its big technology firms were given a platform to compete.

Microsoft is a long way from being a voice of reason, if not a voice of reason. For all its


Generate conditional samples using GPT2. Enter a prompt to have GPT2 generate text related to any topic.

In [ ]:
# Input text
input_text = input() # enter a prompt here...

# Tokenize Input
input_ids = tokenizer.encode(input_text, return_tensors='tf')

# Generate output
outputs = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=100, 
    top_p=0.80, 
    top_k=0
)

print("Generated text:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The Professor
Generated text:
The Professor is determined to be a multi-award winning professor and an accomplished collegiate runner.

The Professor is an accomplished collegiate runner and an accomplished collegiate runner. The Professor's personal growth has been enabled by his incredible inner clarity and personality.

The Professor's Personal Growth Has Been Enabled by His Amazing Inner Truths and the Outward Attitude of His Center (Zafar).

This project was funded by the Daniel K. Krantz Foundation, which will be holding
